In [95]:
import pandas as pd
import json
import random
import csv

In [96]:
# Constantes

In [97]:
# Ruta de los archivos
PATH = "../"
coco_file = PATH + "Data/JSONs/dataset_caltech_coco.json"
csv_file = PATH + "Data/CSVs/dataset_ampliado_caltech.csv"

In [98]:
# Cargar el archivo JSON del dataset
with open(coco_file, 'r') as f:
    coco_data = json.load(f)

# Cargar el archivo CSV
df = pd.read_csv(csv_file)

In [99]:
# Crear un diccionario para mapear el path de las imágenes a sus anotaciones
image_annotations = {}
for annotation in coco_data["annotations"]:
    image_id = annotation["image_id"]
    bbox = annotation["bbox"]
    category_id = annotation["category_id"]
    image_info = coco_data["images"][image_id - 1]  # Restamos 1 para obtener el índice correcto
    image_path = image_info["dir_name"] + "/" + image_info["file_name"]
    if image_path in image_annotations:
        image_annotations[image_path].append([category_id, bbox[0], bbox[1], bbox[2], bbox[3]])
    else:
        image_annotations[image_path] = [[category_id, bbox[0], bbox[1], bbox[2], bbox[3]]]

In [100]:
# Agregar las bounding boxes al DataFrame
bounding_boxes = []
for index, row in df.iterrows():
    image_path = row["path"]
    if row["class"] == "empty":
        bounding_boxes.append([])
    elif image_path in image_annotations:
        bboxes = image_annotations[image_path]
        bounding_boxes.append(bboxes)
    else:
        bounding_boxes.append([])

df["bounding_boxes"] = bounding_boxes

In [103]:
# Guardar el DataFrame resultante en un nuevo archivo CSV
output_csv_file = PATH + "Data/CSVs/dataset_caltech_bb.csv"
# Quotes are added to strings column in case they contain the delimiter character (or other CSV formatting characters). -> boundingboxes with "
df.to_csv(output_csv_file, index=False)